In [ ]:
import tensorflow as tf


import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import collections
import random
import re 
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [ ]:
embedding_dim = 256
BUFFER_SIZE = 1000
units = 512
features_shape  = 2048
attention_features_shape = 64

In [ ]:
# This model is an image caption model using GRU and attention to generates caption.
# This file contains the implementation of the model components only.
# It uses the encoder-decoder architecture to generate captions.
# The encoder uses a pretrained CNN model to extract features from image before feeding it to a fully connected layer that reduces its dimensions
# The encoder output is processed by the decoder using an attention mechanism that weights the features before it is finally processed by a GRU
# and two fully connected layers to generate captions.


class Attention(tf.keras.Model):
  # Attention mechanism implementation
  def __init__(self, units):
    super(Attention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape = (batch_size  , hidden_size)
    # hidden_with_time_axis_shape ==  (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden,1)

    #attention_hidden_layer shape == (batch_size, 64, units)
    attention_hidden_layer  = (tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 64, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights_shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size ) / ( batch_size , 64 ,1)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class CNN_Encoder(tf.keras.Model):
  # Since features have already been extracted and dumped it using pickle
  # This encoder passes those features through a fully connected layer
  def __init__(self, embedding_dim):
    super(CNN_Encoder, self).__init__()
    # shape after fc == (batch_size, 64, embedding_dim)
    self.fc = tf.keras.layers.Dense(embedding_dim)

  def call(self,x):
    x = self.fc(x)
    x = tf.nn.relu(x)
    return x

In [ ]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.GRU = tf.keras.layers.GRU(self.units,
                                   return_sequences = True,
                                   return_state = True,
                                   recurrent_initializer = 'glorot_uniform')
    
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention  = Attention(self.units)

  def call(self,x, features ,hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.GRU(x)

    # x shape == (batch_size , max_length, hidden_size) / (batch_size,1,64)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state ,attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))



In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits =True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss = loss_object(real,pred)

  mask  = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
def checkpoint_manager(encoder, decoder, optimizer,path = './checkpoints/train'):
  ckpt = tf.train.Checkpoint(encoder = encoder,
                             decoder = decoder,
                             optimizer = optimizer)
  ckpt_manager = tf.train.CheckpointManager(ckpt, path, max_to_keep=5)

  return ckpt , ckpt_manager

In [ ]:
def restore_checkpoint(ckpt, ckpt_manager):
  start_epoch = 0
  if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
    features = encoder(img_tensor)

    for i in range(1, target.shape[1]):
      # passing the features through the decoder
      predictions, hidden, _ = decoder(dec_input, features , hidden)

      loss += loss_function(target[:, i], predictions)

      #using teacher forcing
      dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss


def train(dataset, num_steps, ckpt_manager, start_epoch= 0, epochs=25):
  # Customized to train from latest checkpoint

  loss_plot = []
  for epoch in range(start_epoch, epochs):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
      batch_loss , t_loss = train_step(img_tensor, target)
      total_loss += t_loss

      if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch +1, batch, batch_loss.numpy() / int(target.shape[1])))

    # storing the epoch end loss value to plot later 
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()
    
    print( 'Epoch {} Loss {:.6f}'.format(epoch + 1, total_loss/ num_steps))

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    # Do num_stepps = train_size/ batch _size

In [ ]:
def plot_loss(loss_plot):
  plt.plot(loss_plot)
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.title('Loss Plot')
  plt.show()




In [ ]:
def load_model():
  # fuction loads the pretrained inception model
  image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

  new_input = image_model.input
  hidden_layer = image_model.layers[-1].output

  image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

  return image_features_extract_model


def load_image(image_path, res_w, res_h, channels=3):
  # function loads image , resizes it to the input dimension shape of the pretrained inception model and then processes the image using the inception model.
  img = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(img, channels= channels)
  img = tf.image.resize(img, (res_w, res_h))
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  return img, image_path

In [ ]:

def generate_caption(image,image_features_extract_model, encoder,decoder, max_length):
  # This function is customized to generate captions using the 'teacher forcing' mechanism.
  attention_plot = np.zeros((max_length, attention_features_shape))

  hidden = decoder.reset_state(batch_size=1)

  temp_input = tf.expand_dims(load_image(image)[0], 0) # Define load_image
  img_tensor_val = image_features_extract_model(temp_input)
  img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

  features = encoder(img_tensor_val) # encoder output

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
  result = []

  for i in range(max_length):
    predictions , hidden , attention_weights  = decoder(dec_input, features, hidden) # decoder output

    attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy() # save attention weights in attention_plot dict

    predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()# get token

    result.append(tokenizer.index_word[predicted_id])

    if tokenizer.index_word[predicted_id] == '<end>':
      return result, attention_plot

    dec_input = tf.expand_dims([predicted_id], 0)

  attention_plot = attention_plot[: len(result), :]
  return result, attention_plot

In [ ]:
def plot_attention(image, result, attention_plot):
  # Function is used to create attention plots on images during caption generation.
  temp_image = np.array(Image.open(image))

  fig = plt.figure(figsize=(10,10))

  len_result = len(result)

  for l in range(len(result)):
    temp_att = np.resize(attention_plot[l], (8,8))
    ax = fig.add_subplot(len_result//2, len_result//2, l+1)
    ax.set_title(result[l])
    img = ax.imshow(temp_image)
    ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

  plt.tight_layout()
  plt.show()